In [33]:
!pip install langchain pinecone-client pypdf huggingface_hub sentence-transformers -q

In [2]:
# create directory
!mkdir pdfs

In [3]:
# load data from directory
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [4]:
data

[Document(page_content='RENGA RAJAN K\nAI/ML Engineer\n♂phone+91 9043450829 /envel⌢pemaharengarajan46@gmail.com /linkedinLinkedIn /githubGitHub\nAbout me\nExperienced AI Engineer with expertise in building and deploying AI products, managing data pipelines, and model\nlifecycle management. Strong skills in Python and Machine Learning concepts. Proven ability to collaborate with\ncross-functional teams and stay current with advancements in AI.\nTechnical Skills\nCore Skills: Python, Flask, Machine Learning, Computer Vision, Natural Language Processing, Generative AI\nAI Libraries: PyTorch, TensorFlow, scikit-learn, numpy, pandas, Hugging Face\nMLOps Tools: git, DVC, GitHub Actions, MLflow, Docker\nCloud: AWS ECR, AWS EC2, AWS S3, AWS Elastic Beanstalk\nDatabase: MySQL, MongoDB\nExperience\nDatanetiix Solutions Inc April 2023 – Present\nAI/ML Engineer Chennai, Tamilnadu\n•Designing, developing, and implementing end-to-end machine learning projects leveraging MLOps techniques. This\ninvol

In [8]:
# chunking process
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=25)
text_chunks = text_splitter.split_documents(data)

In [9]:
len(text_chunks)

20

In [10]:
# embedding
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
query_result = embeddings.embed_query("my name is raja")

In [13]:
len(query_result)

384

In [22]:
# initialize pinecone
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '4f72adde-3d6a-4327-a78c-0fbd481f35f1')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1')

In [23]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [24]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

In [27]:
! pip install langchain_pinecone -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 3.4 MB/s eta 0:00:00


In [28]:
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [29]:
vectorstore_from_texts = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name='test',
        embedding=embeddings,
    )

In [31]:
# similarity search
query = "Who is Renga Rajan K?"
docs = vectorstore_from_texts.similarity_search(query)

In [32]:
len(docs) #by default gives 4 output

4

In [34]:
# now load LLm model
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [35]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [36]:
# make a chain
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint

In [37]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=20, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [39]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=vectorstore_from_texts.as_retriever(),
                                  return_source_documents=True)

In [42]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [46]:

# full example
query = "who is Renga Rajan K?"
llm_response = qa_chain(query)
# output = process_llm_response(llm_response)

In [47]:
print(llm_response)

{'query': 'who is Renga Rajan K?', 'result': " Renga Rajan K is an experienced AI Engineer with a background in building and deploying AI products, managing data pipelines, and implementing end-to-end machine learning projects using MLOps techniques. He has previously worked as a Machine Learning Engineer at Foxconn Hon Hai Tech in Chennai, Tamilnadu, and has a educational background from P.A.C Ramasamy Raja Polytechnic College in Rajapalayam. He can be reached at +91 9043450829, [pemaharengarajan46@gmail.com](mailto:pemaharengarajan46@gmail.com), and on LinkedIn and GitHub under the name 'Renga Rajan K'.", 'source_documents': [Document(page_content='P.A.C Ramasamy Raja Polytechnic College, Rajapalayam\n2012 - 2015'), Document(page_content='RENGA RAJAN K\nAI/ML Engineer\n♂phone+91 9043450829 /envel⌢pemaharengarajan46@gmail.com /linkedinLinkedIn /githubGitHub\nAbout me\nExperienced AI Engineer with expertise in building and deploying AI products, managing data pipelines, and model'), Do

In [48]:
# full example
query = "is rengarajan k is an AI/ML engineer?"
llm_response = qa_chain(query)
print(llm_response)

{'query': 'is rengarajan k is an AI/ML engineer?', 'result': ' Yes, Rengarajan K is an AI/ML Engineer based on the information provided. He has mentioned his expertise in building and deploying AI products, managing data pipelines, and model training in his about me section. Additionally, he has listed his previous role as a Machine Learning Engineer in his work experience.', 'source_documents': [Document(page_content='RENGA RAJAN K\nAI/ML Engineer\n♂phone+91 9043450829 /envel⌢pemaharengarajan46@gmail.com /linkedinLinkedIn /githubGitHub\nAbout me\nExperienced AI Engineer with expertise in building and deploying AI products, managing data pipelines, and model'), Document(page_content='AI/ML Engineer Chennai, Tamilnadu\n•Designing, developing, and implementing end-to-end machine learning projects leveraging MLOps techniques. This\ninvolves developing robust pipelines for data processing, model training, and deployment.'), Document(page_content='Foxconn Hon Hai Tech Sep 2021 – April 2023\

In [50]:
# full example
query = "where is he currently working?"
llm_response = qa_chain(query)
print(llm_response)

{'query': 'where is he currently working?', 'result': ' He is currently working as an AI/ML Engineer in Chennai, Tamilnadu.', 'source_documents': [Document(page_content='RENGA RAJAN K\nAI/ML Engineer\n♂phone+91 9043450829 /envel⌢pemaharengarajan46@gmail.com /linkedinLinkedIn /githubGitHub\nAbout me\nExperienced AI Engineer with expertise in building and deploying AI products, managing data pipelines, and model'), Document(page_content='industry trends to inform logistics planning.\nPfizer Healthcare India May 2015 – Feb 2019\nJunior Officer Chennai, Tamilnadu\n•Developed and adhered to monthly maintenance plans, tracked equipment breakdowns, diagnosed root causes, and\nimplemented corrective measures.'), Document(page_content='P.A.C Ramasamy Raja Polytechnic College, Rajapalayam\n2012 - 2015'), Document(page_content='AI/ML Engineer Chennai, Tamilnadu\n•Designing, developing, and implementing end-to-end machine learning projects leveraging MLOps techniques. This\ninvolves developing rob